In [4]:
%load_ext autoreload
%autoreload 2
from yahpo_train.cont_normalization import ContNormalization
from yahpo_train.model  import *
from yahpo_train.metrics import *
from yahpo_gym.benchmarks import lcbench, rbv2
from yahpo_gym.configuration import cfg

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
cc = cfg('rbv2_glmnet')
dls = dl_from_config(cc, bs=2048)

In [5]:
f = FFSurrogateModel(dls, layers=[512,512])
l = SurrogateTabularLearner(dls, f, loss_func=nn.MSELoss(reduction='mean'), metrics=nn.MSELoss)
l.metrics = [AvgTfedMetric(mae),  AvgTfedMetric(r2), AvgTfedMetric(spearman)]
l.add_cb(MixHandler)
l.add_cb(EarlyStoppingCallback(patience=3))

In [6]:
l.fit_flat_cos(5, 1e-3)
for p in l.model.wide.parameters():
    p.requires_grad = False
l.fit_flat_cos(5, 1e-4)

epoch,train_loss,valid_loss,mae,r2,spearman,time


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
l.export_onnx(cfg)

c:\Users\flo\Documents\yahpo_gym\yahpo_train\yahpo_train\cont_normalization.py:50: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if torch.abs(lmbda) < self.eps:
c:\Users\flo\Documents\yahpo_gym\yahpo_train\yahpo_train\cont_normalization.py:56: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if torch.abs(lmbda - 2.) <= self.eps:
c:\Users\flo\Documents\yahpo_gym\yahpo_train\yahpo_train\model.py:138: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value wil

In [18]:
a = torch.Tensor([1, np.nan, 2, 2, 1])
b = torch.mode(a[~torch.isnan(a)]).values
torch.nan_to_num(a, b)
b

tensor(1.)